In [195]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn.preprocessing import LabelEncoder
from PIL import Image
import numpy as np
import os
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing import image

In [ ]:
IMAGE_PATH = '../data/images'
IMG_HEIGHT, IMG_WIDTH = 224, 224
BATCH_SIZE = 32
EPOCHS = 100

In [ ]:
# Preprocessing function
def img_preprocessing(path):
    image = Image.open(path)
    image = image.resize((IMG_HEIGHT, IMG_WIDTH)) # Resize the image to (224, 224)
    image = image.convert('RGB') # Convert the image to RGB (in case it's grayscale or has an alpha channel)
    image = np.array(image) / 255.0
    
    return image

In [198]:
# Initialize lists to store images and labels
images = []
labels = []

# Loop through each class folder
for class_name in os.listdir(IMAGE_PATH):
    class_dir = os.path.join(IMAGE_PATH, class_name)
    if not os.path.isdir(class_dir):
        continue
    
    # Loop through each image in the class folder
    for image_name in os.listdir(class_dir):
        image_path = os.path.join(class_dir, image_name)
        
        # Preprocess the image
        img = img_preprocessing(image_path)
        
        # Append the image and its label to the lists
        images.append(img)
        labels.append(class_name)  # Use the folder name as the label

# Convert lists to numpy arrays
images = np.array(images)
labels = np.array(labels)

In [ ]:
# Encode labels
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

In [200]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.25, random_state=42)

In [201]:
# Create an ImageDataGenerator for data augmentation and normalization
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    rescale=1./255
)

In [ ]:
# Load EfficientNetB0 with pre-trained weights, excluding the top layers
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))

In [203]:
# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(len(labels), activation='softmax')(x)

In [204]:
# Combine the base model and the custom layers
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

In [205]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [206]:
# Learning rate scheduler
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6)

In [209]:
# Train the model
history = model.fit(
    datagen.flow(X_train, y_train, batch_size=BATCH_SIZE),
    validation_data=(X_test, y_test),
    callbacks=[lr_scheduler],
    epochs=EPOCHS
)

MemoryError: Unable to allocate 2.27 GiB for an array with shape (4050, 224, 224, 3) and data type float32